In [ ]:
#Import Library
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from google.colab import drive
drive.mount('/content/drive')
#Load Data Set
df = pd.read_csv('/content/drive/My Drive/Tugas_Besar_Machine_Learning-Keompok2/3. Dataset-Kelompok2.csv')


df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [ ]:
from scipy.stats import zscore
df['tau1 Z'] = zscore(df['tau1'])
df['tau2 Z'] = zscore(df['tau2'])
df['tau3 Z'] = zscore(df['tau3'])
df['tau4 Z'] = zscore(df['tau4'])

df['p1 Z'] = zscore(df['p1'])
df['p2 Z'] = zscore(df['p2'])
df['p3 Z'] = zscore(df['p3'])
df['p4 Z'] = zscore(df['p4'])

df['g1 Z'] = zscore(df['g1'])
df['g2 Z'] = zscore(df['g2'])
df['g3 Z'] = zscore(df['g3'])
df['g4 Z'] = zscore(df['g4'])


In [ ]:
lbl = {'unstable': 0, 'stable': 1}
df.stabf = df.stabf.astype('str').map(lbl)

In [ ]:
X = df.drop(['tau1','tau2','tau3','tau4','p1', 'p2', 'p3', 'p4','g1','g2','g3','g4','stab','stabf'], axis=1)
y = df['stabf']
X.head()

,tau1 Z,tau2 Z,tau3 Z,tau4 Z,p1 Z,p2 Z,p3 Z,p4 Z,g1 Z,g2 Z,g3 Z,g4 Z
0,-0.835374,-0.791317,1.141704,1.652103,0.017397,1.079405,-0.017078,-1.092545,0.457467,1.220013,1.321628,1.579026
1,1.478297,-0.126705,-0.803111,-1.415043,1.752124,-1.593619,-1.438158,-0.011575,-0.406791,1.230354,0.135424,0.936256
2,1.357093,1.312140,-0.803499,-1.471504,-0.458492,0.098253,-0.062840,0.760963,-1.319852,0.881299,1.146596,-1.513802
3,-1.653138,0.882289,-0.278354,-1.060901,0.284250,0.513904,-1.591046,0.583414,-0.287304,1.647250,1.474543,-0.591750
4,-0.771543,0.860108,-0.111670,1.680114,-0.298075,0.287450,-1.376343,1.606636,0.992226,-0.253610,0.481133,1.079063


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 0)
X_train_res, y_train_res = sm.fit_resample(X, y)

print(X_train_res.value_counts())
print(y_train_res.value_counts())

tau1 Z     tau2 Z     tau3 Z     tau4 Z     p1 Z       p2 Z       p3 Z       p4 Z       g1 Z       g2 Z       g3 Z       g4 Z     
-1.731763   0.541261   0.012527   0.199741  -0.481848   1.148567   0.640203  -0.951823   1.012636  -1.169639   0.427580  -0.042991    1
 0.502901   1.054685  -0.065832  -0.382308  -1.010268   1.295710   0.498479  -0.039404   1.483580  -0.203235  -1.262697   0.696619    1
 0.503421  -1.057941   0.752767  -0.313268   0.362612   0.724527   0.189991  -1.544357   0.115140   1.426917  -1.147612   0.911770    1
 0.503862  -0.796830   0.066779  -0.759408  -1.654781   0.521356   1.037272   1.315643  -1.297411   1.253407   0.690435  -0.001311    1
 0.504028   1.529795   1.471245  -0.434561   1.049079  -1.112816   0.886128  -1.595511   0.247335  -0.625154  -0.161553   1.026025    1
                                                                                                                                     ..
-0.713592  -0.820028   0.611879   1.003422   0.541789

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res, test_size = 0.3, random_state = 0)

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 6,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)

# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X_train_res ,label=y_train_res)

**xgb_clf**.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)

In [ ]:
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
print(f"XGBOOSTING")
print(f"Accuracy    : {accuracy_score(y_test, y_pred)}")
print(f"Precision   : {precision_score(y_test, y_pred)}")
print(f"Recall      : {recall_score(y_test, y_pred)}")
print(f"F-Measure   : {f1_score(y_test, y_pred)}")

XGBOOSTING
Accuracy    : 0.9566353187042842
Precision   : 0.9461262185736276
Recall      : 0.967979002624672
F-Measure   : 0.9569278671510121


In [ ]:
from sklearn.svm import SVC
svm = SVC(verbose=True, kernel='rbf', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

[LibSVM]

In [ ]:

print(f"SVM")
print(f"Accuracy    : {accuracy_score(y_test, y_pred_svm)}")
print(f"Precision   : {precision_score(y_test, y_pred_svm)}")
print(f"Recall Score: {recall_score(y_test, y_pred_svm)}")
print(f"F-Measure   : {f1_score(y_test, y_pred_svm)}")

SVM
Accuracy    : 0.9579414838035528
Precision   : 0.9381909547738694
Recall Score: 0.9800524934383202
F-Measure   : 0.9586649550706033


In [ ]:
>>> from sklearn.ensemble import RandomForestClassifier
>>> clf=RandomForestClassifier()
>>> clf.fit(X_train, y_train)
y_pred_rdf=clf.predict(X_test)

In [ ]:
print(f"Random Forrest")
print(f"Accuracy    : {accuracy_score(y_test, y_pred_rdf)}")
print(f"Precision   : {precision_score(y_test, y_pred_rdf)}")
print(f"Recall Score: {recall_score(y_test, y_pred_rdf)}")
print(f"F-Measure   : {f1_score(y_test, y_pred_rdf)}")

Random Forrest
Accuracy    : 0.9396551724137931
Precision   : 0.9206030150753769
Recall Score: 0.9616797900262467
F-Measure   : 0.9406931964056483
